#Importing necessary libraries


In [ ]:
!pip install snscrape
import pandas as pd                           #For reading and creating csv files and working with dataframes
import snscrape.modules.twitter as sntwitter  #For utilizing the web scraper in order to get tweets regarding a company
import numpy as np                            #Basic library to perform linear algebra operations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 361 kB/s 


In [ ]:
!pip install transformers requests beautifulsoup4 numpy pandas    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.6 MB/s 
     |████████████████████████████████| 182 kB 79.0 MB/s 


#Importing Transformer
We will be using pretrained BERT base Multilingual model which has been fine tuned for sentiment classification

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification      #Importing tokenizer and classifier for our tweets analysis
import torch                                                                    #Importing pytorch to work with tensors and transformer
import re                                                                       #Importing regex to read thorugh text and get essential data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

#Example of using Tokenizer and getting sentiment 

In [ ]:
import time

In [ ]:
%%time
tokens = tokenizer.encode('I liked the movie a lot', return_tensors='pt')

CPU times: user 672 µs, sys: 942 µs, total: 1.61 ms
Wall time: 3.36 ms


In [ ]:
%%time
result = model(tokens)

CPU times: user 74.8 ms, sys: 3.26 ms, total: 78 ms
Wall time: 175 ms


In [ ]:
%%time
result.logits

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


tensor([[-2.6136, -1.6030,  0.7919,  1.9232,  1.0566]],
       grad_fn=<AddmmBackward0>)

In [ ]:
%%time
int(torch.argmax(result.logits))+1

CPU times: user 966 µs, sys: 185 µs, total: 1.15 ms
Wall time: 4.87 ms


4

#Sentiment analysis on tweets
First we created a function in order to simplif the process of getting the sentiment of a tweet. It takes the text as input and gives out the sentiment score form 1 to 5

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

Web scraper to get tweets based on a query

Here we are trying to get tweets related to TESLA

In [ ]:
query = "#tesla OR #teslastocks OR #teslacars"   #tags realted to which we need our tweets
tweets =[]
public_tweets =  enumerate(sntwitter.TwitterSearchScraper(query).get_items())
from nltk.sentiment.vader import SentimentIntensityAnalyzer
df = pd.DataFrame()
data = []

for i, tweet in public_tweets:
     if(i>100):
       break
     else:
        data.append([tweet.date, tweet.content])
df = pd.DataFrame(data,columns=['date','Text'])

In [ ]:
df

,date,Text
0,2022-12-30 10:17:37+00:00,"The year in tech, by tech guru Chriet 🤓\n\nhtt..."
1,2022-12-30 10:16:48+00:00,Charging #Tesla #Model3 in Brenner on our way ...
2,2022-12-30 10:16:28+00:00,#ETTD\n\nElon Musk’s net worth collapse is big...
3,2022-12-30 10:15:57+00:00,@sspencer_smb @EPA @MartinViecha Biden hates T...
4,2022-12-30 10:14:42+00:00,Mit -34% steuert der #Nasdaq Composite Index a...
...,...,...
96,2022-12-30 09:06:54+00:00,Zabıta 1 Ton Küflenmiş Patates Ele Geçirdi\n\n...
97,2022-12-30 09:05:05+00:00,"Nem állt meg a rendőröknek a Tesla - kiderült,..."
98,2022-12-30 09:04:57+00:00,Tesla is ending the year on a bump after its s...
99,2022-12-30 09:04:15+00:00,#Tesla will announce the prices of the Model S...


In [ ]:
df['sentiment'] = df['Text'].apply(lambda x: sentiment_score(x[:280]))
df

,date,Text,sentiment
0,2022-12-30 10:17:37+00:00,"The year in tech, by tech guru Chriet 🤓\n\nhtt...",1
1,2022-12-30 10:16:48+00:00,Charging #Tesla #Model3 in Brenner on our way ...,5
2,2022-12-30 10:16:28+00:00,#ETTD\n\nElon Musk’s net worth collapse is big...,1
3,2022-12-30 10:15:57+00:00,@sspencer_smb @EPA @MartinViecha Biden hates T...,1
4,2022-12-30 10:14:42+00:00,Mit -34% steuert der #Nasdaq Composite Index a...,1
...,...,...,...
96,2022-12-30 09:06:54+00:00,Zabıta 1 Ton Küflenmiş Patates Ele Geçirdi\n\n...,1
97,2022-12-30 09:05:05+00:00,"Nem állt meg a rendőröknek a Tesla - kiderült,...",1
98,2022-12-30 09:04:57+00:00,Tesla is ending the year on a bump after its s...,1
99,2022-12-30 09:04:15+00:00,#Tesla will announce the prices of the Model S...,1


In [ ]:
df.to_csv('tweets.csv')   #Saving into a csv file for further use